In [1]:
import numpy as np
import pointMass
import gym

In [4]:
data = np.load("collected_data/20000HER2_pointMassObject-v0_Hidden_256l_2.npz")

In [5]:
replan_interval = 15
lower_achieved_whole_state = True

data = np.load("collected_data/20000HER2_pointMassObject-v0_Hidden_256l_2.npz")
obs = data['obs']
ags = data['achieved_goals']
if lower_achieved_whole_state:
    higher_level_acts = data['full_positional_states']
    #act_dim_higher = env.observation_space.spaces['full_positional_state'].shape[0]
else:
    higher_level_acts = data['controllable_achieved_goal']
    #act_dim_higher = env.observation_space.spaces['controllable_achieved_goal'].shape[0]
lower_level_acts = data['acts']

In [6]:

# ok, what do we want to do?
# Q1 - DO we want to learn from one long thing, or multiple demos?
# lets say we want to multiple demos.
# then what are we doing? Sample random start and end, sample at intervals of replan_interval.
# desired goal for higher level is the achieved goal of the final state oberveed
# we want to create observation / action pairs of each set of obs separated by the replan interval
# action is either the whole state, or just the controllable section depending on the flag.

def sample_relay_batch(obs, ags, higher_level_acts, lower_level_acts, replan_interval = 15):
    # first thing we need to do is sample a couple of trajectories
    num_trajectories = 10
    traj_indexes = np.random.choice(obs.shape[0], num_trajectories)

    # then, in each trajectory take a random start and end
    max_start_index = int(obs.shape[1]*0.2) # the start index can be anywhere in the first 20% of steps. 
    end_indices = np.arange(int(obs.shape[1]*0.8), obs.shape[1]) # the end indice can be anywhere in the last 20% of steps
    traj_start_indices = np.random.choice(max_start_index, num_trajectories)
    traj_end_indices = np.random.choice(end_indices, num_trajectories)

    # first things first, get it working using iterators - we can always optimise into matrix form later. 
    high_in_array = []
    high_out_array = []
    low_in_array = []
    low_out_array = []

    traj_obs = obs[traj_indexes]
    for t in range(0,num_trajectories):
        traj_obs = obs[traj_indexes][t]
        traj_higher_level_acts = higher_level_acts[traj_indexes][t]
        traj_ags = ags[traj_indexes][t]
        traj_acts = lower_level_acts[traj_indexes][t]
        replan_time_steps = np.arange(traj_start_indices[t],traj_end_indices[t],replan_interval)
        # we want the higher level observations to be the obs at each higher level step
        # we want the higher level action to be the achieved_goal of the corresponding next higher level step
        # thats why we take 1: onwards for one, and up to the last one for the other.
        high_obs = traj_obs[replan_time_steps[:-1]]
        high_acts = traj_higher_level_acts[replan_time_steps[1:]]
        # we also want the desired goal of the higher level, which is just the last ag
        goal = traj_ags[-1]
        # now tile it out so we have one for each higher level obs to later concat along the last dimension
        high_desired_goals = np.tile(goal,[len(high_obs),1])
        high_in = np.concatenate([high_obs, high_desired_goals], axis=-1)
        # now, the lower level. We want each lower level ob to have desired goal of the corresponding next higher level act.
        # obs is still obs.
        # act is the baseline act. 
        # sample a lower_level obsfrom somewhere within the replan time window of each higher level step.
        low_obs_indexes = replan_time_steps - np.random.choice(replan_interval, len(replan_time_steps))
        low_obs = traj_obs[low_obs_indexes]
        low_goals = traj_higher_level_acts[replan_time_steps]
        low_acts = traj_acts[low_obs_indexes]
        low_in = np.concatenate([low_obs, low_goals], axis = -1)


        # add to the arrays
        high_in_array.append(high_in)
        high_out_array.append(high_acts)
        low_in_array.append(low_in)
        low_out_array.append(low_acts)

    return np.concatenate(high_in_array), np.concatenate(high_out_array), np.concatenate(low_in_array), np.concatenate(low_out_array)
    
    
    
    
    
    # then, between the start and end take a transition every replan_interval steps
    
    
    
high_in, high_out, low_in, low_out = sample_relay_batch(obs, ags, higher_level_acts, lower_level_acts, replan_interval)


In [30]:
data.files

['acts',
 'obs',
 'desired_goals',
 'achieved_goals',
 'controllable_achieved_goals',
 'full_positional_states']

In [6]:
env.close()

closing


In [2]:
env = gym.make('pointMass-v0')
env.render(mode='human')
env.reset()

seeding
Environment set to sparse reward


{'observation': array([2.0300024, 2.99292  , 0.       , 0.       ], dtype=float32),
 'achieved_goal': array([2.0300024, 2.99292  ], dtype=float32),
 'desired_goal': array([ 1.0095205 , -0.76843745], dtype=float32),
 'extra_info': None,
 'controllable_achieved_goal': array([2.0300024, 2.99292  ], dtype=float32),
 'full_positional_state': array([2.0300024, 2.99292  ], dtype=float32),
 'image': array([[[236, 236, 236],
         [234, 234, 234],
         [232, 232, 232],
         ...,
         [227, 227, 227],
         [227, 227, 227],
         [227, 227, 227]],
 
        [[235, 235, 235],
         [233, 233, 233],
         [232, 232, 232],
         ...,
         [227, 227, 227],
         [227, 227, 227],
         [227, 227, 227]],
 
        [[235, 235, 235],
         [233, 233, 233],
         [232, 232, 232],
         ...,
         [227, 227, 227],
         [227, 227, 227],
         [227, 227, 227]],
 
        ...,
 
        [[227, 227, 227],
         [227, 227, 227],
         [227, 227, 

In [3]:
replan_interval = 15
lower_achieved_whole_state = True
#data = np.load("collected_data/20000HER2_pointMassObject-v0_Hidden_256l_2.npz")
data = np.load("collected_data/10000HER2_pointMass-v0_Hidden_256l_2.npz")
obs = data['obs']
ags = data['achieved_goals']
if lower_achieved_whole_state:
    higher_level_acts = data['full_positional_states']
    #act_dim_higher = env.observation_space.spaces['full_positional_state'].shape[0]
else:
    higher_level_acts = data['controllable_achieved_goals']
    #act_dim_higher = env.observation_space.spaces['controllable_achieved_goal'].shape[0]
lower_level_acts = data['acts']

In [4]:
# first thing we need to do is sample a couple of trajectories
num_trajectories = 10
traj_indexes = np.random.choice(obs.shape[0], num_trajectories)

# then, in each trajectory take a random start and end
max_start_index = int(obs.shape[1]*0.2) # the start index can be anywhere in the first 20% of steps. 
end_indices = np.arange(int(obs.shape[1]*0.8), obs.shape[1]) # the end indice can be anywhere in the last 20% of steps
traj_start_indices = np.random.choice(max_start_index, num_trajectories)
traj_end_indices = np.random.choice(end_indices, num_trajectories)

# first things first, get it working using iterators - we can always optimise into matrix form later. 
high_in_array = []
high_out_array = []
low_in_array = []
low_out_array = []

t = 0
traj_obs = obs[traj_indexes][t]
traj_higher_level_acts = higher_level_acts[traj_indexes][t]
traj_ags = ags[traj_indexes][t]
traj_acts = lower_level_acts[traj_indexes][t]
replan_time_steps = np.arange(traj_start_indices[t],traj_end_indices[t],replan_interval)
# we want the higher level observations to be the obs at each higher level step
    # we want the higher level action to be the achieved_goal of the corresponding next higher level step
# thats why we take 1: onwards for one, and up to the last one for the other.
high_obs = traj_obs[replan_time_steps[:-1]]
high_acts = traj_higher_level_acts[replan_time_steps[1:]]
# we also want the desired goal of the higher level, which is just the last ag
goal = traj_ags[-1]
# now tile it out so we have one for each higher level obs to later concat along the last dimension
high_desired_goals = np.tile(goal,[len(high_obs),1])
high_in = np.concatenate([high_obs, high_desired_goals], axis=-1)
# now, the lower level. We want each lower level ob to have desired goal of the corresponding next higher level act.
# obs is still obs.
# act is the baseline act. 
# sample a lower_level obsfrom somewhere within the replan time window of each higher level step.
low_obs_indexes = replan_time_steps - np.random.choice(replan_interval, len(replan_time_steps))
low_obs = traj_obs[low_obs_indexes]
low_goals = traj_higher_level_acts[replan_time_steps]
low_acts = traj_acts[low_obs_indexes]
low_in = np.concatenate([low_obs, low_goals], axis = -1)



In [5]:
import time
env.reset_goal_pos(goal)
for i in range(0,150):
    o = traj_obs[i]
    env.initialize_start_pos(o) 
    ag = traj_ags[i]
    sub_g = traj_higher_level_acts[i+replan_interval]
    
    sub_g2 = traj_ags[i+replan_interval]
    
    env.visualise_sub_goal(sub_g, lower_achieved_whole_state)
    time.sleep(0.05)

initing


IndexError: index 100 is out of bounds for axis 0 with size 100

In [23]:
def visualise_sub_goal(self, sub_goal, lower_achieved_whole_state):
    

    # in the sub_goal case we either only  have the positional info, or we have the full state positional info.
    #print(sub_goal)
    index = 0
    if self.sub_goals is None:
        self.sub_goals = []
        self.sub_goal_cids = []
        print('initing')
        sphereRadius = 0.3
        mass = 1
        colSphereId = self._p.createCollisionShape(p.GEOM_SPHERE, radius=sphereRadius)
        relativeChildPosition = [0, 0, 0]
        relativeChildOrientation = [0, 0, 0, 1]
        alpha = 0.5
        colors = [[212/250,175/250,55/250,alpha], [1,1,0,alpha], [0,0,1,alpha]]


        for g in range(0, len(sub_goal)//2):
            if g == 0:
                # the sphere
                visId = p.createVisualShape(p.GEOM_SPHERE, radius = sphereRadius,
                                                 rgbaColor=colors[g])
            else:
                visId = p.createVisualShape(p.GEOM_BOX, halfExtents=[0.35,0.35,0.35],
                                                 rgbaColor=colors[g])

            self.sub_goals.append(self._p.createMultiBody(mass, colSphereId, visId, [sub_goal[index], sub_goal[index + 1], 0.1]))
            collisionFilterGroup = 0
            collisionFilterMask = 0
            self._p.setCollisionFilterGroupMask(self.sub_goals[g], -1, collisionFilterGroup, collisionFilterMask)
            self.sub_goal_cids.append(
                self._p.createConstraint(self.sub_goals[g], -1, -1, -1, self._p.JOINT_FIXED, [sub_goal[index], sub_goal[index + 1], 0.1], [0, 0, 0.1],
                                         relativeChildPosition, relativeChildOrientation))
            index +=2


    else:

        for g in range(0, len(sub_goal)//2):
            
            self._p.resetBasePositionAndOrientation(self.sub_goals[g], [sub_goal[index], sub_goal[index + 1], 0.1],
                                                    [0, 0, 0, 1])
            self._p.changeConstraint(self.sub_goal_cids[g], [sub_goal[index], sub_goal[index + 1], 0.1], maxForce=100)
            index += 2

    
env.visualise_sub_goal = visualise_sub_goal

In [9]:
env.test()

5


In [25]:
data.files

['acts',
 'obs',
 'desired_goals',
 'achieved_goals',
 'controllable_achieved_goals',
 'full_positional_states']

In [8]:
np.concatenate([data['obs'], data['desired_goals']], axis = -1).shape

(10, 100, 6)

In [15]:
indexes = np.random.choice(data['obs'].shape[0], 512)

In [13]:
indexes.shape

(512,)

In [14]:
indexes

array([1, 5, 9, 3, 6, 1, 3, 7, 5, 5, 9, 5, 2, 1, 5, 5, 9, 5, 9, 0, 8, 0,
       5, 3, 7, 0, 9, 0, 2, 5, 1, 8, 6, 6, 5, 8, 9, 1, 0, 1, 4, 3, 0, 3,
       5, 8, 0, 7, 2, 1, 4, 2, 6, 6, 8, 5, 9, 2, 1, 7, 6, 5, 2, 4, 6, 1,
       4, 7, 4, 3, 1, 9, 0, 9, 0, 6, 3, 6, 6, 5, 4, 3, 8, 8, 8, 6, 8, 4,
       8, 0, 8, 3, 6, 1, 2, 2, 3, 4, 6, 5, 2, 4, 4, 8, 3, 4, 8, 1, 9, 9,
       6, 1, 1, 0, 9, 4, 4, 6, 6, 8, 9, 8, 3, 3, 0, 7, 9, 7, 5, 0, 6, 8,
       2, 0, 4, 7, 2, 7, 1, 0, 9, 7, 6, 1, 7, 0, 7, 6, 7, 2, 0, 1, 3, 7,
       0, 1, 6, 4, 5, 0, 3, 7, 5, 2, 6, 3, 1, 7, 5, 0, 3, 8, 2, 3, 1, 9,
       7, 0, 7, 3, 1, 6, 1, 2, 9, 5, 7, 6, 2, 9, 3, 9, 7, 7, 7, 9, 0, 0,
       1, 5, 0, 1, 6, 7, 2, 9, 0, 4, 7, 8, 6, 7, 4, 9, 5, 6, 2, 5, 9, 9,
       5, 8, 1, 7, 2, 2, 6, 1, 5, 0, 6, 5, 5, 3, 6, 6, 8, 7, 1, 2, 7, 5,
       8, 6, 5, 6, 6, 4, 4, 4, 5, 0, 5, 0, 0, 7, 3, 2, 1, 5, 7, 1, 8, 1,
       3, 5, 4, 4, 3, 3, 3, 1, 3, 5, 0, 5, 8, 9, 5, 6, 9, 1, 4, 8, 6, 6,
       6, 6, 9, 3, 4, 8, 8, 5, 5, 7, 6, 7, 7, 8, 1,

In [18]:
indexes = np.random.choice(5, [10,3])

In [19]:
indexes.shape

(10, 3)

In [20]:
indexes

array([[2, 0, 3],
       [3, 4, 1],
       [0, 2, 2],
       [2, 3, 0],
       [1, 4, 0],
       [0, 1, 0],
       [0, 1, 2],
       [1, 1, 4],
       [2, 3, 1],
       [1, 3, 2]])

In [24]:
obs = data['obs']
acts = data['acts']

train_length = int(0.8 * (len(obs)))
train_obs,train_acts  = obs[:train_length, :,:], acts[:train_length, :,:]
valid_obs,  valid_acts  = obs[train_length:, :,:], acts[train_length:, :,:]

In [27]:
sequences = np.random.choice(len(train_obs), 5)
print(sequences)

[4 3 0 4 6]


In [28]:
obs[sequences].shape

(5, 100, 4)

In [36]:
np.concatenate(obs,axis=0).shape

(1000, 4)